In [1]:
try:
    import sys
    import os
    sys.path.insert(0, os.path.abspath(os.curdir))
except ModuleNotFoundError:
    pass
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium
import carregar_mapa_desagrupados as cmd
import carregar_dados_agrupados as cda


<p>1 - Iniciar sessão spark</p>

In [2]:
spark_session = cda.iniciar_sessao_spark()

23/09/30 16:46:12 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/30 16:46:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/30 16:46:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataframe_desagrupados = cmd.load_dataframe_desagrupados(spark=spark_session)
dataframe_desagrupados.show()

+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|            CONSOCIO|             EMPRESA|DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|           LATITUDE|          LONGITUDE|DATA_EXTRACAO_API|TURNO|
+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   2023-09-15|   08:50|          3054-10|               2|                     33743|         47399|         2023-09-15T11:50:12Z|         -23.598638|         -46.416157|       2023-09-15|Manhã|


In [4]:
df_prefixo_onibus = cmd.obter_prefixo_onibus(dataframes_desagrupados_completo=dataframe_desagrupados)
df_prefixo_onibus

[66580,
 15173,
 73951,
 78598,
 26486,
 61679,
 36102,
 31944,
 66070,
 11567,
 48280,
 22129,
 31703,
 15057,
 26543,
 77605,
 36103,
 45298,
 55474,
 66114,
 63212,
 31156,
 64442,
 71655,
 32098,
 68463,
 56242,
 52743,
 72935,
 52001,
 10422,
 61749,
 71530,
 48899,
 51418,
 21965,
 63676,
 52910,
 51428,
 64317,
 31762,
 11745,
 51056,
 72485,
 15194,
 45166,
 72926,
 45410,
 25207,
 16530,
 55426,
 45726,
 71429,
 16597,
 85760,
 52051,
 71285,
 71292,
 47745,
 71386,
 81572,
 36077,
 47050,
 71549,
 16673,
 81443,
 48963,
 52904,
 76085,
 77441,
 71548,
 45390,
 26140,
 85641,
 15221,
 68166,
 68047,
 63181,
 63448,
 86153,
 66911,
 73916,
 71255,
 51105,
 68752,
 76050,
 11771,
 47741,
 48203,
 63678,
 45724,
 63586,
 81038,
 55329,
 55093,
 77434,
 11860,
 45558,
 68314,
 51112,
 72928,
 31159,
 32204,
 47683,
 15133,
 11988,
 31111,
 47636,
 63109,
 47229,
 78203,
 52745,
 22808,
 31126,
 68608,
 45134,
 31102,
 80978,
 31832,
 36577,
 45473,
 16499,
 22707,
 45580,
 45474,


In [5]:
prefixo_onibus = '68712'

In [6]:
dataframe_filter = cmd.dataframe_filter_desagrupados(
    dataframes_desagrupados_completo=dataframe_desagrupados,
    data_extracao='2023-09-18',
    prefixo_onibus=prefixo_onibus,
    turno='Noite'
)
dataframe_filter.show()

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|   18:00|         68712|                     33040|          546T-10|        -23.6541035|-46.710938999999996|
|   18:05|         68712|                     33040|          546T-10|       -23.65393525|-46.707456500000006|
|   18:10|         68712|                     33040|          546T-10|        -23.6543535|      -46.701634625|
|   18:15|         68712|                     33040|          546T-10|       -23.65908975|        -46.7012395|
|   18:20|         68712|                     33040|          546T-10|        -23.6613215|         -46.693996|
|   18:25|         68712|                     33040|          546T-10|         -23.670195|         -46.688707|
|

In [7]:
letreiro_onibus = cmd.obter_letreiro_onibus(dataframe_prefixo_onibus=dataframe_filter)
letreiro_onibus

'546T-10'

In [9]:
rota_linha = cmd.obter_rota_linha(letreiro_onibus=letreiro_onibus)
rota_linha[0]

['546T-10', 'Vl. Guacuri - Term. Sto. Amaro', '0082BA']